In [51]:
import os
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from PIL import Image
from PIL.Image import FLIP_LEFT_RIGHT, FLIP_TOP_BOTTOM, ROTATE_90, ROTATE_180, ROTATE_270
from PIL.ImageEnhance import Color, Contrast, Brightness, Sharpness
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import np
from torch.autograd import Variable
from torchvision import transforms

In [3]:
IMG_PATH = '/data/amazon/train-jpg/'
IMG_EXT = '.jpg'
TRAIN_DATA = '/data/amazon/train_v2.csv'

In [30]:
class KaggleAmazonDataset(Dataset):
    def __init__(self, csv_path, img_path, img_ext, transform=None):
        tmp_df = pd.read_csv(csv_path)
        check_file = lambda x: os.path.isfile(img_path+x+img_ext)
        assert tmp_df['image_name'].apply(check_file).all(), \
                                "Some images referenced in CSV file were not found"
            
        self.mlb = MultiLabelBinarizer()
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform
        self.X_train = tmp_df['image_name']
        self.y_train = self.mlb.fit_transform(tmp_df['tags'].str.split()).astype(np.float32)
        
    def __getitem__(self, index):
        img = Image.open(self.img_path + self.X_train[index] + self.img_ext)
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
            
        label = torch.from_numpy(self.y_train[index])
        return img, label
    
    def __len__(self):
        return len(self.X_train.index)

In [31]:
transformations = transforms.Compose([transforms.Scale(32), transforms.ToTensor()])
dset_train = KaggleAmazonDataset(TRAIN_DATA, IMG_PATH, IMG_EXT, transformations)

In [58]:
train_loader = DataLoader(dset_train, batch_size=64, shuffle=True, num_workers=8)

In [59]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(2304, 256)
        self.fc2 = nn.Linear(256, 17)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)

In [60]:
model = Net().cuda()

In [61]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [64]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(async=True), target.cuda(async=True)
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.binary_cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [66]:
for epoch in range(5):
    train(epoch)

Train Epoch: 0 [0/40479 (0%)]	Loss: 0.272938
Train Epoch: 0 [640/40479 (2%)]	Loss: 0.269210
Train Epoch: 0 [1280/40479 (3%)]	Loss: 0.268473
Train Epoch: 0 [1920/40479 (5%)]	Loss: 0.275899
Train Epoch: 0 [2560/40479 (6%)]	Loss: 0.251330
Train Epoch: 0 [3200/40479 (8%)]	Loss: 0.271356
Train Epoch: 0 [3840/40479 (9%)]	Loss: 0.284028
Train Epoch: 0 [4480/40479 (11%)]	Loss: 0.285575
Train Epoch: 0 [5120/40479 (13%)]	Loss: 0.304178
Train Epoch: 0 [5760/40479 (14%)]	Loss: 0.297058
Train Epoch: 0 [6400/40479 (16%)]	Loss: 0.274520
Train Epoch: 0 [7040/40479 (17%)]	Loss: 0.267491
Train Epoch: 0 [7680/40479 (19%)]	Loss: 0.289976
Train Epoch: 0 [8320/40479 (21%)]	Loss: 0.239947
Train Epoch: 0 [8960/40479 (22%)]	Loss: 0.237798
Train Epoch: 0 [9600/40479 (24%)]	Loss: 0.286251
Train Epoch: 0 [10240/40479 (25%)]	Loss: 0.246125
Train Epoch: 0 [10880/40479 (27%)]	Loss: 0.304064
Train Epoch: 0 [11520/40479 (28%)]	Loss: 0.299875
Train Epoch: 0 [12160/40479 (30%)]	Loss: 0.313110
Train Epoch: 0 [12800/40479

Train Epoch: 2 [24960/40479 (62%)]	Loss: 0.294757
Train Epoch: 2 [25600/40479 (63%)]	Loss: 0.257831
Train Epoch: 2 [26240/40479 (65%)]	Loss: 0.256311
Train Epoch: 2 [26880/40479 (66%)]	Loss: 0.271017
Train Epoch: 2 [27520/40479 (68%)]	Loss: 0.278332
Train Epoch: 2 [28160/40479 (70%)]	Loss: 0.259724
Train Epoch: 2 [28800/40479 (71%)]	Loss: 0.251373
Train Epoch: 2 [29440/40479 (73%)]	Loss: 0.283209
Train Epoch: 2 [30080/40479 (74%)]	Loss: 0.248427
Train Epoch: 2 [30720/40479 (76%)]	Loss: 0.248617
Train Epoch: 2 [31360/40479 (77%)]	Loss: 0.287481
Train Epoch: 2 [32000/40479 (79%)]	Loss: 0.247844
Train Epoch: 2 [32640/40479 (81%)]	Loss: 0.261878
Train Epoch: 2 [33280/40479 (82%)]	Loss: 0.220742
Train Epoch: 2 [33920/40479 (84%)]	Loss: 0.285308
Train Epoch: 2 [34560/40479 (85%)]	Loss: 0.235855
Train Epoch: 2 [35200/40479 (87%)]	Loss: 0.247217
Train Epoch: 2 [35840/40479 (88%)]	Loss: 0.212335
Train Epoch: 2 [36480/40479 (90%)]	Loss: 0.250278
Train Epoch: 2 [37120/40479 (92%)]	Loss: 0.266033


In [6]:
class AmazonDataset(Dataset):
    def __init__(self, csv_path, img_path, img_ext, transform=None):
        self.df = pd.read_csv(csv_path)
        func = lambda x: os.path.isfile(img_path + x + img_ext)
        assert self.df['image_name'].apply(func).all(), 'some images in csv file not found'
        
        self.mlb = MultiLabelBinarizer(classes=['clear', 'cloudy', 'haze','partly_cloudy',
                                                'agriculture','artisinal_mine',
                                                'bare_ground','blooming','blow_down',
                                                'conventional_mine','cultivation',
                                                'habitation','primary','road',
                                                'selective_logging','slash_burn','water'])
        
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform
        self.X = self.df['image_name']
        self.y = self.mlb.fit_transform(self.df['tag'].str.split()).astype(np.float32)
    
    def __getitem__(self, index):
        img = Image.open(self.img_path + self.X[index] + self.img_ext)
        img = img.convert('RGB')
        if self.transform is not None: img = self.transform(img)
        label = from_numpy(self.y[index])
        return img, label
    
    def __len__(self): 
        return len(self.df.index)
    
    def getLabelEncoder(self): 
        return self.mlb
    
    def getDF(self): 
        return self.df
    
    def X(self): 
        return self.X